<a href="https://colab.research.google.com/github/onuralpArsln/MlAiTutorialProjects/blob/main/ImageClass/CarPlane/imageClasser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prep

Eğer colab ile açtıysan mevcut colab workspaceine data kalsöürnü çekmen lazım neyseki bu çok kolay sadece ağaıdaki komutları kullan

In [5]:
!wget https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CarPlane/data.zip

--2024-07-16 13:11:33--  https://github.com/onuralpArsln/MlAiTutorialProjects/raw/main/ImageClass/CarPlane/data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CarPlane/data.zip [following]
--2024-07-16 13:11:34--  https://raw.githubusercontent.com/onuralpArsln/MlAiTutorialProjects/main/ImageClass/CarPlane/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4850435 (4.6M) [application/zip]
Saving to: ‘data.zip.1’

data.zip.1          100%[===================>]   4.62M  --.-KB/s    in 0.03s   

2024-07-16 13:11:34 (177 MB/s) - ‘data.zip.1

In [6]:
!unzip data.zip

Archive:  data.zip
replace data/test/cars/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace data/test/cars/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A N
  inflating: data/test/cars/10.jpg   
  inflating: data/test/cars/11.jpg   
  inflating: data/test/cars/12.jpg   
  inflating: data/test/cars/13.jpg   
  inflating: data/test/cars/14.jpg   
  inflating: data/test/cars/15.jpg   
  inflating: data/test/cars/16.jpg   
  inflating: data/test/cars/17.jpg   
  inflating: data/test/cars/18.jpg   
  inflating: data/test/cars/19.jpg   
  inflating: data/test/cars/2.jpg    
  inflating: data/test/cars/20.jpg   
  inflating: data/test/cars/21.jpg   
  inflating: data/test/cars/22.jpg   
  inflating: data/test/cars/23.jpg   
  inflating: data/test/cars/24.jpg   
  inflating: data/test/cars/25.jpg   
  inflating: data/test/cars/26.jpg   
  inflating: data/test/cars/27.jpg   
  inflating: data/test/cars/28.jpg   
  inflating: data/test/cars/29.jpg   
  inflating: data/test/cars/3.jpg    
  

# Classification Model With Keras


Bu çalışmada Keras kullanarak bir sınıflandırma yapacağız.

Data klasörü içine bakarak kullanacağımız görsellere bakabilirsin.

Toplamda 500 adet görsel mevcuttur.


200 uçak ve 200 araba görseli modeli eğitmek için 50 uçak ve 50 araba görseli modeli test etmek için kullanılacak.


In [7]:
# keras versiyonuna göre yorum bu altaki iki importtan birini yapman gerekebilir
#from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# keras versiyonuna göre yorum bu yukardaki iki importtan birini yapman
# yorum satırını değiştirmeyi dene hata alırsan

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

img_width, img_height = 224, 224


görüntüler 224 pixele 224 pixel. Sınıflandırma için süper hd bir görüntüye gerek yok. Görüntülerde hızlıca çalışmak sınıflandırma için daha verimli olacak.

değişkenlerimize bilgileri atıyoruz.

In [8]:
train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16


Epoch ve Batch_size önemli kavramlar  

Epoch basitçe veri seti üzerinde kça iterasyon yapılacağını ifade eder

Batch size ise bir seferde incelenecek örnek sayısını ifade eder.


In [9]:
if K.image_data_format() == 'channels_first':
	input_shape = (3, img_width, img_height)
else:
	input_shape = (img_width, img_height, 3)


Burada amaç veri formatının düzgün aktarılmasından emin olmak. Channels Rgb kanalları, eğer rgb kanalları ilk yazılır ise önce kanal sonra boyut verilir. Aksi halde önce boyut sonra kanal ifade ediliyor.

In [10]:
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))


Burada aslında ağ mimarisini yazdık. daha ileri seviye çalışmalar için mevcut başarılı mimariler kullanmak güzeldir ama ne olduğu görmek için önemli bi adım    



Conv2D , görüntüyü birden fazla görüntüye evriştiren katmandır.


Aktivasyon , aktivasyon fonksiyonudur.
MaxPooling2D, verilen boyut matrisindeki değeri maksimuma çıkarmak için kullanılır ve aynısı sonraki 2 katman için kullanılır. daha sonra Düzleştir , evriştirildikten sonra elde edilen görüntünün boyutlarını düzleştirmek için kullanılır.
Yoğun, bunu tamamen bağlantılı bir model haline getirmek için kullanılır ve gizli katmandır.
Bırakma, veri kümesine aşırı sığmayı önlemek için kullanılır.
Yoğun, çıktı katmanının görüntünün hangi kategoriye ait olduğuna karar veren yalnızca bir nöron içermesidir.

In [11]:
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])


Burada kayıp, optimize ediciler ve metriklerin kullanımını içeren derleme işlevi kullanılmıştır. Burada kullanılan kayıp fonksiyonu Binary_crossentropy, kullanılan optimizer ise rmsprop'tur .

In [12]:
# eğer aşağıda pil ile ilgili hata alaırsan bunu kullan
!pip3 install pillow

In [13]:
import sys
from PIL import Image
sys.modules['Image'] = Image
from PIL import Image
print(Image.__file__)
import Image
print(Image.__file__)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py
/usr/local/lib/python3.10/dist-packages/PIL/Image.py


In [14]:



train_datagen = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
	train_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
	validation_data_dir,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

model.fit(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)


Found 400 images belonging to 2 classes.
Found 100 images belonging to 2 classes.
Epoch 1/10
25/25 [==============================] - 39s 1s/step - loss: 0.7017 - accuracy: 0.5725 - val_loss: 0.5826 - val_accuracy: 0.5625
Epoch 2/10
25/25 [==============================] - 23s 920ms/step - loss: 0.5191 - accuracy: 0.7550 - val_loss: 0.3350 - val_accuracy: 0.8438
Epoch 3/10
25/25 [==============================] - 22s 860ms/step - loss: 0.4782 - accuracy: 0.7975 - val_loss: 0.2765 - val_accuracy: 0.9062
Epoch 4/10
25/25 [==============================] - 23s 891ms/step - loss: 0.4231 - accuracy: 0.8250 - val_loss: 0.3546 - val_accuracy: 0.8333
Epoch 5/10
25/25 [==============================] - 24s 953ms/step - loss: 0.3259 - accuracy: 0.8500 - val_loss: 0.2953 - val_accuracy: 0.8750
Epoch 6/10
25/25 [==============================] - 23s 924ms/step - loss: 0.3815 - accuracy: 0.8425 - val_loss: 0.2989 - val_accuracy: 0.8542
Epoch 7/10
25/25 [==============================] - 29s 1s/step

Şimdi dataGenerator'ın kısmı devreye giriyor. Hangisinde kullandık:

Görüntüyü yeniden ölçeklendiren, belirli bir aralıkta kesme uygulayan, görüntüyü yakınlaştıran ve görüntüyle yatay çevirme yapan ImageDataGenerator . Bu ImageDataGenerator görüntünün tüm olası yönelimlerini içerir.
train_datagen.flow_from_directory, train_dataset dizininden veri hazırlamak için kullanılan işlevdir Target_size, görüntünün hedef boyutunu belirtir.
test_datagen.flow_from_directory , model için test verilerini hazırlamak için kullanılır ve hepsi yukarıdakine benzer.
fit_generator, verileri yukarıda yapılan modele sığdırmak için kullanılır; kullanılan diğer faktörler,step_per_epochs'tur ve bize modelin eğitim verileri için kaç kez yürütüleceğini bildirir.
Epochs bize modelin ileri ve geri geçişte kaç kez eğitileceğini söyler.
validation_data, doğrulama/test verilerini modele beslemek için kullanılır.
validation_steps doğrulama/test örneklerinin sayısını belirtir.

In [21]:
model.save('model_saved.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [23]:
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16
import numpy as np

from keras.models import load_model

model = load_model('model_saved.h5')

image = load_img('data/test/planes/5.jpg', target_size=(224, 224))
img = np.array(image)
img = img / 255.0
img = img.reshape(1,224,224,3)
label = model.predict(img)
print("Predicted Class (0 - Cars , 1- Planes): ", label[0][0])


1/1 [==============================] - 0s 126ms/step
Predicted Class (0 - Cars , 1- Planes):  0.8610517
